In [37]:
import pandas as pd
from math import sqrt
import csv

In [2]:
'''
Para o nosso caso eu já separei duas bases de dados com os dados para treinar o nosso algoritmo e uma base para
testarmos. Porém, o comum é usar uma base e dividi-la em uma certa porcentagem (ex.: 80.0% para teste - 20.0% para
treino) alocando os dados aleatóriamente nas duas bases. 
Então porque decidimos serpar nós mesmos as duas bases?
Porque na nossa base de treinamento todas as respostas das 10 questões tendem a classe na qual aquela linha foi
classificada. Assim, acredita-se que obteremos uma melhor classificação de qual classe o individuo está mais 
próximo.
'''

'\nPara o nosso caso eu já separei duas bases de dados com os dados para treinar o nosso algoritmo e uma base para\ntestarmos. Porém, o comum é usar uma base e dividi-la em uma certa porcentagem (ex.: 80.0% para teste - 20.0% para\ntreino) alocando os dados aleatóriamente nas duas bases. \nEntão porque decidimos serpar nós mesmos as duas bases?\nPorque na nossa base de treinamento todas as respostas das 10 questões tendem a classe na qual aquela linha foi\nclassificada. Assim, acredita-se que obteremos uma melhor classificação de qual classe o individuo está mais \npróximo.\n'

In [ ]:
'''
Função usada para carregar os dados dos arquivos CSV
'''

In [12]:
def load_csv(filename):
    dataset = list()
    with open(filename, 'r') as file:
        csv_reader = csv.reader(file)
        for row in csv_reader:
            if not row:
                continue
            dataset.append(row)
        return dataset

In [13]:
train_data = load_csv("./training.csv")
test_data = load_csv("./testing.csv")

In [5]:
'''
Vamos usar essa função para calcular as distancias entre os pontos para que possamos agrupar em clusters
e classificar os nossos dados.
A a função matemática escolhida aqui para o nosso caso foi a a Distância euclidiana. Porém, existem outras
como por exemplo a distância de Manhatan, distância Mahalanobis, etc.
Agora vamos para a função!!
A função distance recebe três parâmetros: X1, X2, data.
    -X1: vetor de informações de um dado qualquer
    -X2: vetor de informações de um dado qualquer
Segue o link para mais informações sobre a Distância de euclidiana:
https://pt.wikipedia.org/wiki/Dist%C3%A2ncia_euclidiana
'''

'\nVamos usar essa função para calcular as distancias entre os pontos para que possamos agrupar em clusters\ne classificar os nossos dados.\nA a função matemática escolhida aqui para o nosso caso foi a a Distância de euclidiana. Porém, existem outras\ncomo por exemplo a distância de Manhatan, distância Mahalanobis, etc.\nAgora vamos para a função!!\nA função distance recebe três parâmetros: X1, X2, data.\n    -X1: vetor de informações de um dado qualquer\n    -X2: vetor de informações de um dado qualquer\n    data: todos vetores incluindo X1 e X2 para podermos calcular a matrix de covariância.\nSegue o link para mais informações sobre a Distância de Mahalanobis:\nhttps://pt.wikipedia.org/wiki/Dist%C3%A2ncia_de_Mahalanobis\n'

In [25]:
def distance(X1, X2):
    sum = 0.0
    for i in range (len(X1)-1):
        sum += (float(X1[i]) - float(X2[i])) * (float(X1[i]) - float(X2[i]))
    return sqrt(sum)
        

In [ ]:
'''
Função auxiliar para transformar as Classes em inteiros
'''

In [15]:
def str_column_to_int(dataset, column):
    class_values = [row[column] for row in dataset]
    unique = set(class_values)
    lookup = dict()
    for i, value in enumerate(unique):
        lookup[value] = i
        print('[%s] => %d' % (value, i))
    for row in dataset:
        row[column] = lookup[row[column]]
    return lookup

In [ ]:
'''
Para evitar qualquer problema de tipo, essa função pode ser usada para converter os dados.
'''

In [16]:
def str_column_to_float(dataset, column):
    for row in dataset:
        row[column] = float(row[column].strip())

In [ ]:
'''
Como o proprio nome já diz, vamos usar essa função para pegar os k vizinhos similares do vetor test na 
matriz train.
'''

In [32]:
def get_neighbors(train, test, k):
    distances = list()
    for row in train:
        dist = distance(row, test)
        distances.append((row, dist))
    distances.sort(key=lambda tup: tup[1])
    neighbors = list()
    for neighbor in range(k):
        neighbors.append(distances[neighbor][0])
    return neighbors

In [18]:
'''
Essa função irá classificar nosso dado de test em uma das classes de treino de acordo com o k
'''

'\n\n'

In [34]:
def prediction_classification(train, test, k):
    neighbors = get_neighbors(train, test, k)
    output_values = [row[-1] for row in neighbors]
    prediction = max(set(output_values), key=output_values.count)
    return prediction

In [ ]:
'''
Dado um conjunto de dados de test vamos realizar a classificação de acordo com os nosso dados de treino chamando a
função KNN
'''

In [35]:
def KNN(train, test, k):
    predictions = list()
    for row in test:
        predictions.append((row, prediction_classification(train, row, k)))
    return predictions

In [40]:
k = 3
print(KNN(train_data, test_data, k))   

[(['1', '2', '4', '4', '2', '2', '1', '4', '2', '1'], 'Corvinal'), (['3', '1', '3', '3', '4', '1', '1', '4', '1', '2'], 'Corvinal'), (['3', '4', '2', '1', '3', '1', '3', '4', '3', '1'], 'Grifinoria'), (['2', '3', '4', '4', '2', '3', '3', '2', '1', '1'], 'Corvinal'), (['4', '4', '1', '1', '4', '2', '1', '2', '2', '1'], 'Grifinoria'), (['4', '4', '4', '4', '2', '3', '2', '4', '3', '3'], 'Corvinal'), (['3', '1', '2', '4', '2', '1', '1', '2', '3', '3'], 'Grifinoria'), (['3', '1', '1', '2', '1', '2', '2', '2', '3', '2'], 'Grifinoria'), (['3', '4', '3', '4', '1', '4', '1', '1', '1', '2'], 'Grifinoria'), (['3', '1', '2', '1', '3', '1', '1', '4', '2', '3'], 'Grifinoria')]


In [ ]:
'''
Vamos usar a biblioteca do pandas para carregar nossas questões em um dataframe com um delibitador diferente
'''

In [19]:
pd.options.display.max_colwidth = 1000
questions = pd.read_csv('./questions.csv', sep=';'  , engine='python')

In [ ]:
'''
O código abaixo é um quiz para saber qual Casa você pertence. Execute todos os passo anteriores em ordem
e depois execute este passo e divirta-se.
'''

In [ ]:
name = input("Seu nome é? ")
answers = list()
for index, row in questions.iterrows():
    print(row)
    answer = "0"
    while answer not in ["1", "2", "3", "4"]:
        answer = input("\nDigite o número de sua resposta: ")
        if answer not in ["1", "2", "3", "4"]:
            print("\nResposta inválida! Selecione uma opção de 1 a 4!\n")
    answers.append(float(answer))
print(answers)
print("\nKNN Seletor: Difícil. Muito difícil. Tem muita coragem, estou vendo. Uma mente nada má também.\n Tem talento, se tem. E uma sede de provar seu valor. \nMas onde vou colocá-lo?")
label = prediction_classification(train_data, answers, k)
str_column_to_int(train_data, len(train_data[0])-1)
print("\nKNN Seletor: Não? Tem certeza? \nVocê poderia ser grande, sabia? Está tudo aqui, na sua cabeça. \nE iria ajudá-lo a alcançar essa grandeza. Não há dúvida sobre isso. Não?")
print("\nBom, se tem certeza... é melhor que seja...\n")
print('\nA partir de hoje você se chamará %s de %s' % (name, label))

In [ ]:
#Ref.:
#https://machinelearningmastery.com/tutorial-to-implement-k-nearest-neighbors-in-python-from-scratch/
#https://aminoapps.com/c/potter-amino-em-portugues/page/blog/teste-das-casas/g7pG_0et6u738kQkMz5kYNRNK7pDjeBze